In [1]:
from espnapi.espn_api.football import League
import pandas as pd
import plotly.express as px
# Init

#fan's - 42936131
#mine - 829963546
league_id = 829963546
year = 2022
league = League(league_id=league_id, year=year)



## Lets get the draft picks from each team

In [2]:
df = pd.DataFrame()

teams = []
players = []
id = []
round_num = []
round_pick = []


for pick in league.draft:
    teams.append(pick.team.team_name)
    players.append(pick.playerName)
    round_num.append(pick.round_num)
    round_pick.append(pick.round_pick)
    id.append(pick.playerId)
    
df['Team'] = teams
df['Player'] = players
df['ID'] = id
df['RoundPick'] = round_pick
df['RoundNum'] = round_num
df.to_csv('Data/team_players.csv')
df.head(10)

,Team,Player,ID,RoundPick,RoundNum
0,P-Town CASH KEVIN,Jonathan Taylor,4242335,1,1
1,Diggy Wit it,Christian McCaffrey,3117251,2,1
2,My Team A Bust,Austin Ekeler,3068267,3,1
3,Eugene Tweakers,Cooper Kupp,2977187,4,1
4,Couv Reigning Champ,Dalvin Cook,3116593,5,1
5,Gorilla Guys,Derrick Henry,3043078,6,1
6,I dont know what im doing,Leonard Fournette,3115364,7,1
7,Cougar Town Zach Wilson,Deebo Samuel,3126486,8,1
8,No Nyugen Will Win,Najee Harris,4241457,9,1
9,Just-in-N-out Drive thru,Justin Jefferson,4262921,10,1


# Get projected points

In [3]:
players = []
expected = []
actual = []
rank = []
nfl_team = []
for team in league.teams:
    for player in team.roster:
        players.append(player.name)
        expected.append(player.projected_total_points)
        actual.append(player.total_points)
        rank.append(player.posRank)
        nfl_team.append(player.proTeam)

points_df = pd.DataFrame()
points_df['Player'] = players
points_df['Expected_Points'] = expected
points_df['Actual_Points'] = actual
points_df['Rank'] = rank
points_df['NFLTeam'] = nfl_team



In [4]:
points_df['Difference'] = points_df['Actual_Points'] - points_df['Expected_Points']

In [5]:
points_df = points_df.sort_values('Difference', ascending = False)
points_df.to_csv('Data/player_differences.csv')

In [6]:
# points_df.groupby('Team').mean().sort_values('Difference', ascending=False).to_csv('Data/grouped_average_differences.csv')
# points_df.groupby('Team').mean().sort_values('Difference', ascending=False)

# Export the CSV

In [7]:
points_df.to_csv('Data/player_points.csv')
points_df.head(10)

,Player,Expected_Points,Actual_Points,Rank,NFLTeam,Difference
144,Jimmy Garoppolo,14.84,149.26,19,SF,134.42
161,Kenyan Drake,13.48,79.20,41,BAL,65.72
139,Jeff Wilson Jr.,50.21,112.40,23,MIA,62.19
94,Geno Smith,148.82,182.76,7,SEA,33.94
28,Marcus Mariota,141.19,172.32,9,ATL,31.13
76,Jamaal Williams,123.09,153.50,13,DET,30.41
162,Eno Benjamin,64.37,86.30,37,HOU,21.93
140,Patriots D/ST,90.39,111.00,1,NE,20.61
93,Jets D/ST,70.84,91.00,4,NYJ,20.16
159,Josh Reynolds,58.02,73.70,61,DET,15.68


In [8]:
combined_df = pd.merge(df, points_df, on = 'Player')
combined_df.head(10)

,Team,Player,ID,RoundPick,RoundNum,Expected_Points,Actual_Points,Rank,NFLTeam,Difference
0,P-Town CASH KEVIN,Jonathan Taylor,4242335,1,1,316.99,112.00,24,IND,-204.99
1,Diggy Wit it,Christian McCaffrey,3117251,2,1,309.94,201.76,3,SF,-108.18
2,My Team A Bust,Austin Ekeler,3068267,3,1,284.73,230.10,1,LAC,-54.63
3,Eugene Tweakers,Cooper Kupp,2977187,4,1,312.91,201.40,5,LAR,-111.51
4,Couv Reigning Champ,Dalvin Cook,3116593,5,1,258.49,156.50,11,MIN,-101.99
5,Gorilla Guys,Derrick Henry,3043078,6,1,264.47,203.06,2,TEN,-61.41
6,I dont know what im doing,Leonard Fournette,3115364,7,1,258.74,154.70,12,TB,-104.04
7,Cougar Town Zach Wilson,Deebo Samuel,3126486,8,1,261.46,130.00,21,SF,-131.46
8,No Nyugen Will Win,Najee Harris,4241457,9,1,265.48,124.96,20,PIT,-140.52
9,Just-in-N-out Drive thru,Justin Jefferson,4262921,10,1,287.80,215.52,4,MIN,-72.28


In [9]:
combined_df.to_csv('Data/total.csv')

In [10]:
combined_df

,Team,Player,ID,RoundPick,RoundNum,Expected_Points,Actual_Points,Rank,NFLTeam,Difference
0,P-Town CASH KEVIN,Jonathan Taylor,4242335,1,1,316.99,112.00,24,IND,-204.99
1,Diggy Wit it,Christian McCaffrey,3117251,2,1,309.94,201.76,3,SF,-108.18
2,My Team A Bust,Austin Ekeler,3068267,3,1,284.73,230.10,1,LAC,-54.63
3,Eugene Tweakers,Cooper Kupp,2977187,4,1,312.91,201.40,5,LAR,-111.51
4,Couv Reigning Champ,Dalvin Cook,3116593,5,1,258.49,156.50,11,MIN,-101.99
...,...,...,...,...,...,...,...,...,...,...
117,My Team A Bust,Tyler Bass,3917232,3,15,131.58,100.00,1,BUF,-31.58
118,Eugene Tweakers,Cowboys D/ST,-16006,4,15,106.49,109.00,2,DAL,2.51
119,Couv Reigning Champ,Harrison Butker,3055899,5,15,145.15,47.00,29,KC,-98.15
120,Cougar Town Zach Wilson,Brian Robinson Jr.,4241474,8,15,102.87,52.90,52,WSH,-49.97


In [11]:
#normalize
combined_df['Normalized_Difference'] = combined_df['Difference'] /combined_df['Difference'].abs().max()
  

In [12]:
combined_df

,Team,Player,ID,RoundPick,RoundNum,Expected_Points,Actual_Points,Rank,NFLTeam,Difference,Normalized_Difference
0,P-Town CASH KEVIN,Jonathan Taylor,4242335,1,1,316.99,112.00,24,IND,-204.99,-0.912933
1,Diggy Wit it,Christian McCaffrey,3117251,2,1,309.94,201.76,3,SF,-108.18,-0.481785
2,My Team A Bust,Austin Ekeler,3068267,3,1,284.73,230.10,1,LAC,-54.63,-0.243297
3,Eugene Tweakers,Cooper Kupp,2977187,4,1,312.91,201.40,5,LAR,-111.51,-0.496615
4,Couv Reigning Champ,Dalvin Cook,3116593,5,1,258.49,156.50,11,MIN,-101.99,-0.454218
...,...,...,...,...,...,...,...,...,...,...,...
117,My Team A Bust,Tyler Bass,3917232,3,15,131.58,100.00,1,BUF,-31.58,-0.140643
118,Eugene Tweakers,Cowboys D/ST,-16006,4,15,106.49,109.00,2,DAL,2.51,0.011178
119,Couv Reigning Champ,Harrison Butker,3055899,5,15,145.15,47.00,29,KC,-98.15,-0.437116
120,Cougar Town Zach Wilson,Brian Robinson Jr.,4241474,8,15,102.87,52.90,52,WSH,-49.97,-0.222544


# Now get their actual players

In [61]:
teams = []
players = []
projected = []
actual = []
stats = []
ranks = []
weeks = []
current_df = pd.DataFrame()
for i in range(1, 17):
    league.load_roster_week(i)
    for team in league.teams:
        for player in team.roster:
            weeks.append(i)
            teams.append(team.team_name)
            players.append(player.name)
            projected.append(player.projected_total_points)
            actual.append(player.total_points)
            ranks.append(player.posRank)

current_df['Team'] = teams
current_df['Player'] = players
current_df['Week'] = weeks
current_df['Rank'] = ranks
current_df['Expected_Points'] = projected
current_df['Actual_Points'] = actual
current_df['Week'] = weeks
current_df['Difference'] = current_df['Actual_Points'] - current_df['Expected_Points']

In [14]:
current_df.to_csv('Data/current_players.csv')

In [15]:
combined_df.groupby('Team').mean()

,ID,RoundPick,RoundNum,Expected_Points,Actual_Points,Rank,Difference,Normalized_Difference
Team,,,,,,,,
Cougar Town Zach Wilson,3.490025e+06,5.916667,7.083333,196.603333,99.983333,27.750000,-96.620000,-0.430302
Couv Reigning Champ,3.295082e+06,5.454545,7.818182,202.986364,115.692727,22.454545,-87.293636,-0.388767
Diggy Wit it,2.472610e+06,5.500000,7.333333,202.690833,116.321667,25.083333,-86.369167,-0.384649
Eugene Tweakers,2.832234e+06,5.384615,7.153846,222.665385,108.844615,32.538462,-113.820769,-0.506906
Gorilla Guys,2.741681e+06,5.500000,6.500000,206.660833,130.061667,26.083333,-76.599167,-0.341138
I dont know what im doing,1.888404e+06,5.090909,6.545455,195.667273,125.383636,19.909091,-70.283636,-0.313012
Just-in-N-out Drive thru,3.010023e+06,4.750000,6.750000,201.440000,145.245000,10.166667,-56.195000,-0.250267
My Team A Bust,3.897771e+06,5.500000,7.500000,202.554167,140.140000,16.583333,-62.414167,-0.277965
No Nyugen Will Win,3.533783e+06,5.500000,8.357143,194.849286,117.860000,22.857143,-76.989286,-0.342876


In [16]:
now_df = current_df.groupby('Team').mean().sort_values('Difference')
now_df

,Rank,Expected_Points,Actual_Points,Difference
Team,,,,
Cougar Town Zach Wilson,30.361624,195.389889,99.102435,-96.287454
Eugene Tweakers,33.604000,189.472320,99.246080,-90.226240
Couv Reigning Champ,27.903704,183.929074,107.403185,-76.525889
Gorilla Guys,27.670498,189.947203,115.083295,-74.863908
P-Town CASH KEVIN,25.041509,190.976642,124.234868,-66.741774
Diggy Wit it,22.132353,172.115625,107.339485,-64.776140
No Nyugen Will Win,26.657795,170.460114,106.327224,-64.132890
I dont know what im doing,22.552124,178.528456,116.290579,-62.237876
My Team A Bust,23.369004,174.430369,118.459779,-55.970590


In [17]:
drafted_df = combined_df.groupby('Team').mean().sort_values('Difference')
drafted_df

,ID,RoundPick,RoundNum,Expected_Points,Actual_Points,Rank,Difference,Normalized_Difference
Team,,,,,,,,
Eugene Tweakers,2.832234e+06,5.384615,7.153846,222.665385,108.844615,32.538462,-113.820769,-0.506906
Cougar Town Zach Wilson,3.490025e+06,5.916667,7.083333,196.603333,99.983333,27.750000,-96.620000,-0.430302
Couv Reigning Champ,3.295082e+06,5.454545,7.818182,202.986364,115.692727,22.454545,-87.293636,-0.388767
Diggy Wit it,2.472610e+06,5.500000,7.333333,202.690833,116.321667,25.083333,-86.369167,-0.384649
No Nyugen Will Win,3.533783e+06,5.500000,8.357143,194.849286,117.860000,22.857143,-76.989286,-0.342876
Gorilla Guys,2.741681e+06,5.500000,6.500000,206.660833,130.061667,26.083333,-76.599167,-0.341138
P-Town CASH KEVIN,2.910000e+06,5.153846,7.769231,216.589231,145.603077,15.230769,-70.986154,-0.316140
I dont know what im doing,1.888404e+06,5.090909,6.545455,195.667273,125.383636,19.909091,-70.283636,-0.313012
My Team A Bust,3.897771e+06,5.500000,7.500000,202.554167,140.140000,16.583333,-62.414167,-0.277965


In [18]:
#Actual x = draft
#Actual y = Now
comparison_df = pd.merge(drafted_df, now_df, on = 'Team')
comparison_df = comparison_df[['Expected_Points_x', 'Actual_Points_x', 'Difference_x', 'Rank_x', 'Expected_Points_y', 'Actual_Points_y', 'Difference_y', 'Rank_y']]



In [19]:
comparison_df['Delta_Projections'] = comparison_df['Difference_x'] - comparison_df['Difference_y']
comparison_df['Delta_Ranks'] = comparison_df['Rank_x'] - comparison_df['Rank_y']
comparison_df = comparison_df.sort_values('Delta_Projections', ascending = False)
comparison_df[['Delta_Projections', 'Delta_Ranks']].sort_values('Delta_Ranks', ascending=False)

,Delta_Projections,Delta_Ranks
Team,,
Diggy Wit it,-21.593027,2.950980
Eugene Tweakers,-23.594529,-1.065538
Gorilla Guys,-1.735259,-1.587165
Cougar Town Zach Wilson,-0.332546,-2.611624
I dont know what im doing,-8.045760,-2.643033
No Nyugen Will Win,-12.856396,-3.800652
Couv Reigning Champ,-10.767747,-5.449158
My Team A Bust,-6.443576,-6.785670
Just-in-N-out Drive thru,-4.620097,-7.524453


In [60]:
current_df

,Team,Player,Rank,Expected_Points,Actual_Points,Difference
0,I dont know what im doing,Leonard Fournette,12,258.74,154.7,-104.04
1,I dont know what im doing,Davante Adams,3,253.59,216.4,-37.19
2,I dont know what im doing,Mike Evans,20,223.32,134.1,-89.22
3,I dont know what im doing,Breece Hall,22,210.66,115.1,-95.56
4,I dont know what im doing,Brandin Cooks,49,215.10,91.6,-123.50
...,...,...,...,...,...,...
2636,P-Town CASH KEVIN,Breece Hall,22,210.66,115.1,-95.56
2637,P-Town CASH KEVIN,Josh Reynolds,61,58.02,73.7,15.68
2638,P-Town CASH KEVIN,Riley Patterson,22,126.77,65.0,-61.77
2639,P-Town CASH KEVIN,Kenyan Drake,41,13.48,79.2,65.72


In [62]:
test_df = current_df.groupby(['Team', 'Week'], as_index = False).mean()
test_df.head()

C:\Users\yangj\AppData\Local\Temp\ipykernel_1876\1717063357.py:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



,Team,Week,Rank,Expected_Points,Actual_Points,Difference
0,Cougar Town Zach Wilson,1,37.500000,192.600625,90.915000,-101.685625
1,Cougar Town Zach Wilson,2,40.529412,189.660000,88.375294,-101.284706
2,Cougar Town Zach Wilson,3,38.352941,191.875294,89.398824,-102.476471
3,Cougar Town Zach Wilson,4,39.058824,190.450588,88.087059,-102.363529
4,Cougar Town Zach Wilson,5,37.176471,195.675294,92.581176,-103.094118


In [64]:
import plotly.io as pio
pio.renderers
plot = px.scatter(test_df, x = 'Week', y = 'Difference', color = 'Team', trendline = 'ols')
plot.show()


ModuleNotFoundError: No module named 'statsmodels'

In [51]:
plot = px.scatter(test_df, x = 'Week', y = 'Difference', color = 'Team', trendline = 'ols')
plot.show()